# TempoQL Demo on Full MIMIC-IV Dataset

Before running, please `cd` into this repo and run `pip install .` in the virtual environment of your choice. This will install required dependencies so that you can run this notebook.

This example will use the MIMIC-IV-OMOP dataset available [here](https://physionet.org/content/mimic-iv-demo-omop/0.9/). Please download it and set the path to it in the `base_path` variable below.

In [9]:
from tempo_ql import GenericDataset, formats, QueryEngine, FileVariableStore
import numpy as np
import os
import pandas as pd
import time

In [2]:
# GCP project in which to run queries - make sure it has access to MIMIC-IV through physionet.org
project_id = "ai-clinician"
# name of a dataset within your project to store temporary results. Required if you plan to subset the data to run queries
scratch_dataset = None
# directory to store temporary variables
variable_store_dir = "mimiciv_data"

# provide your Gemini API key here if you'd like to try the LLM-assisted authoring workflow
gemini_api_key = None

In [3]:
# Initialize query engine and variable store
dataset = GenericDataset(f'bigquery://{project_id}', formats.mimiciv(), 
                        scratch_schema_name=f'{project_id}.{scratch_dataset}' if scratch_dataset is not None else None)

if not os.path.exists(variable_store_dir): os.mkdir(variable_store_dir)
var_store = FileVariableStore(variable_store_dir)
query_engine = QueryEngine(dataset, variable_stores=[var_store])


In [7]:
# get available concepts
query_engine.dataset.list_data_elements(scope="chartevents", return_counts=True).head(20)

,id,name,count,scope,type
1613,227969,Safety Measures,12869833,chartevents,event
1070,220045,Heart Rate,8752069,chartevents,event
2283,220210,Respiratory Rate,8636655,chartevents,event
2031,220277,O2 saturation pulseoxymetry,8567015,chartevents,event
866,220048,Heart Rhythm,7947289,chartevents,event
765,224650,Ectopy Type 1,7049295,chartevents,event
1622,220179,Non Invasive Blood Pressure systolic,5378740,chartevents,event
1924,220180,Non Invasive Blood Pressure diastolic,5377689,chartevents,event
772,220181,Non Invasive Blood Pressure mean,5372922,chartevents,event
171,227958,Less Restrictive Measures,5021996,chartevents,event


In [ ]:
# Get all visit occurrence IDs
query_engine.get_ids()

In [6]:
start = time.time()
result = query_engine.query("first ((time({id in (220181, 220052); scope = chartevents}) where #value < (mean {id in (220181, 220052); scope = chartevents} from #now - 8 h to #now)) - #now) from #now to #now + 24 h every 24 h")
print(len(result), "results,", time.time() - start, "seconds")

ValueError: Exception occurred when processing variable 'None': 'now' keyword can only be used within a time-series expression, ending with an 'at'/'every'/'at every' clause.

In [10]:
import pandas_gbq

df = pandas_gbq.read_gbq("""
    SELECT * FROM `physionet-data.mimiciv_3_1_hosp.admissions`
    LIMIT 100
""", project_id=project_id)
df

Downloading: 100%|██████████|


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10106244,26713233,2147-05-09 10:34:00,2147-05-12 13:43:00,NaT,DIRECT EMER.,None,PHYSICIAN REFERRAL,HOME,Private,English,SINGLE,WHITE,NaT,NaT,0
1,13700703,20448599,2172-09-25 01:01:00,2172-10-03 13:25:00,NaT,OBSERVATION ADMIT,None,EMERGENCY ROOM,HOME,Private,English,MARRIED,WHITE,2172-09-24 17:38:00,2172-09-25 03:07:00,0
2,15443666,27961368,2168-12-30 23:30:00,2169-01-05 16:02:00,NaT,OBSERVATION ADMIT,None,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,English,SINGLE,BLACK/AFRICAN AMERICAN,2168-12-30 11:19:00,2168-12-31 01:22:00,0
3,16299919,26977065,2193-05-15 08:37:00,2193-05-17 16:03:00,NaT,OBSERVATION ADMIT,None,EMERGENCY ROOM,HOSPICE,Medicare,English,WIDOWED,BLACK/AFRICAN AMERICAN,2193-05-15 04:36:00,2193-05-15 14:27:00,0
4,14149715,24191358,2181-10-25 19:37:00,2181-10-29 14:38:00,NaT,OBSERVATION ADMIT,P00230,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,English,SINGLE,WHITE,2181-10-25 08:48:00,2181-10-26 15:18:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15383419,26624935,2143-08-17 23:22:00,2143-08-22 19:31:00,NaT,EW EMER.,P00628,EMERGENCY ROOM,HOME,Private,English,MARRIED,BLACK/AFRICAN AMERICAN,2143-08-17 16:11:00,2143-08-18 05:57:00,0
96,15423116,24686077,2173-04-17 23:50:00,2173-04-22 00:00:00,2173-04-22 00:00:00,EW EMER.,P00628,EMERGENCY ROOM,DIED,Other,Thai,SINGLE,ASIAN - SOUTH EAST ASIAN,2173-04-17 21:47:00,2173-04-18 02:03:00,1
97,15438386,21324831,2162-12-14 18:49:00,2162-12-15 16:45:00,NaT,EW EMER.,P00628,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,Kabuverdianu,MARRIED,BLACK/CAPE VERDEAN,2162-12-14 11:54:00,2162-12-14 20:21:00,0
98,15696349,21845305,2185-01-20 13:24:00,2185-01-29 11:30:00,2185-01-29 11:30:00,EW EMER.,P00628,EMERGENCY ROOM,DIED,Private,English,SINGLE,BLACK/AFRICAN AMERICAN,2185-01-20 08:49:00,2185-01-20 16:11:00,1
